## DIA 133: Insertar un Resumen Visual Automático en el Excel Exportado

Hoy vas a aprender a incluir gráficos directamente en el archivo Excel que se genera con pandas y xlsxwriter, para ofrecer un resumen visual completo al usuario.

Esto permitirá que, al descargar su historial, obtenga gráficos integrados como:

📊 Barras por categoría

📈 Tendencias por fecha

📋 Estadísticas básicas

🧩 1. Preparar el DataFrame con los reportes
Ya lo tienes en la ruta /exportar-historial.xlsx, pero ahora generaremos también un gráfico de barras de categorías.

Modifica la ruta así:

python
Copiar
Editar
import pandas as pd
from io import BytesIO
from flask import send_file
from collections import Counter

@app.route('/exportar-historial.xlsx')
@login_required
def exportar_historial_excel():
    query = request.args.get('q', '').strip()
    categoria_filtro = request.args.get('categoria', '').strip()

    reportes_query = Reporte.query.filter_by(user_id=current_user.id)

    if query:
        reportes_query = reportes_query.filter(
            db.or_(
                Reporte.comentario.ilike(f'%{query}%'),
                Reporte.categoria.ilike(f'%{query}%'),
                Reporte.url.ilike(f'%{query}%')
            )
        )

    if categoria_filtro:
        reportes_query = reportes_query.filter(Reporte.categoria.ilike(f'%{categoria_filtro}%'))

    reportes = reportes_query.order_by(Reporte.fecha_generado.desc()).all()

    # Crear DataFrame principal
    df = pd.DataFrame([{
        'ID': r.id,
        'Fecha': r.fecha_generado.strftime('%Y-%m-%d'),
        'Tipo': r.tipo,
        'Categoría': r.categoria or 'Sin categoría',
        'Comentario': r.comentario or '',
        'URL': r.url
    } for r in reportes])

    # Contar categorías
    categoria_counter = Counter(df['Categoría'])

    # Crear DataFrame para gráfico
    df_categorias = pd.DataFrame({
        'Categoría': list(categoria_counter.keys()),
        'Cantidad': list(categoria_counter.values())
    })

    # Crear archivo Excel
    output = BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Historial')
        df_categorias.to_excel(writer, index=False, sheet_name='Resumen')

        workbook  = writer.book
        worksheet = writer.sheets['Resumen']

        # Crear gráfico de barras
        chart = workbook.add_chart({'type': 'column'})
        chart.add_series({
            'name':       'Cantidad por Categoría',
            'categories': ['Resumen', 1, 0, len(df_categorias), 0],
            'values':     ['Resumen', 1, 1, len(df_categorias), 1],
            'fill':       {'color': 'green'}
        })

        chart.set_title({'name': 'Distribución por Categoría'})
        chart.set_x_axis({'name': 'Categoría'})
        chart.set_y_axis({'name': 'Cantidad'})
        chart.set_style(11)

        worksheet.insert_chart('D2', chart)

    output.seek(0)
    return send_file(
        output,
        as_attachment=True,
        download_name='historial_reportes_con_grafico.xlsx',
        mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    )